### Evaluating RAG: Using LLM as a Judge (With Structured Outputs)

This cookbook shows an example of using the Mistral AI models for LLM As A Judge using structured outputs.

In [1]:
import os
from pydantic import BaseModel, Field


from enum import Enum
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI

from rich.console import Console

console = Console()

In [2]:
OUTPUT_FILE = "05.5_output.md"

In [3]:
# For variation use a function to get the LLM client based on the user's choice


def get_llm_client(llm_choice):

    if llm_choice == "GROQ":

        client = OpenAI(
            base_url="https://api.groq.com/openai/v1",
            api_key=os.environ.get("GROQ_API_KEY"),
        )

        return client

    elif llm_choice == "OPENAI":

        load_dotenv()  # load environment variables from .env fil

        client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

        return client

    else:

        raise ValueError("Invalid LLM choice. Please choose 'GROQ' or 'OPENAI'.")

In [4]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
load_dotenv(find_dotenv(), override=True)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

LLM_CHOICE = "OPENAI"
# LLM_CHOICE = "GROQ" # GROWQ uses different API

if OPENAI_API_KEY:
    print(f"OPENAI_API_KEY exists and begins {OPENAI_API_KEY[:14]}...")
else:
    print("OPENAI_API_KEY not set")

if GROQ_API_KEY:
    print(f"GROQ_API_KEY exists and begins {GROQ_API_KEY[:14]}...")
else:
    print("GROQ_API_KEY not set")


client = get_llm_client(LLM_CHOICE)
if LLM_CHOICE == "GROQ":
    MODEL = "llama-3.3-70b-versatile"
else:
    MODEL = "gpt-4o-mini"

print(f"LLM_CHOICE: {LLM_CHOICE} - MODEL: {MODEL}")

OPENAI_API_KEY exists and begins sk-proj-vIt-L1...
GROQ_API_KEY exists and begins gsk_0yKDCuUXkz...
LLM_CHOICE: OPENAI - MODEL: gpt-4o-mini


## Main Code For LLM As A Judge For RAG (With Structured Outputs)

In [5]:
# Define Enum for scores
class Score(str, Enum):
    no_relevance = "0"
    low_relevance = "1"
    medium_relevance = "2"
    high_relevance = "3"


# Define a constant for the score description
SCORE_DESCRIPTION = (
    "Score as a string between '0' and '3'. "
    "0: No relevance/Not grounded/Irrelevant - The context/answer is completely unrelated or not based on the context. "
    "1: Low relevance/Low groundedness/Somewhat relevant - The context/answer has minimal relevance or grounding. "
    "2: Medium relevance/Medium groundedness/Mostly relevant - The context/answer is somewhat relevant or grounded. "
    "3: High relevance/High groundedness/Fully relevant - The context/answer is highly relevant or grounded."
)


# Define separate classes for each criterion with detailed descriptions
class ContextRelevance(BaseModel):
    explanation: str = Field(
        ...,
        description=(
            "Step-by-step reasoning explaining how the retrieved context aligns with the user's query. "
            "Consider the relevance of the information to the query's intent and the appropriateness of the context "
            "in providing a coherent and useful response."
        ),
    )
    score: Score = Field(..., description=SCORE_DESCRIPTION)


class AnswerRelevance(BaseModel):
    explanation: str = Field(
        ...,
        description=(
            "Step-by-step reasoning explaining how well the generated answer addresses the user's original query. "
            "Consider the helpfulness and on-point nature of the answer, aligning with the user's intent and providing valuable insights."
        ),
    )
    score: Score = Field(..., description=SCORE_DESCRIPTION)


class Groundedness(BaseModel):
    explanation: str = Field(
        ...,
        description=(
            "Step-by-step reasoning explaining how faithful the generated answer is to the retrieved context. "
            "Consider the factual accuracy and reliability of the answer, ensuring it is grounded in the retrieved information."
        ),
    )
    score: Score = Field(..., description=SCORE_DESCRIPTION)


class RAGEvaluation(BaseModel):
    context_relevance: ContextRelevance = Field(
        ...,
        description="Evaluation of the context relevance to the query, considering how well the retrieved context aligns with the user's intent.",
    )
    answer_relevance: AnswerRelevance = Field(
        ...,
        description="Evaluation of the answer relevance to the query, assessing how well the generated answer addresses the user's original query.",
    )
    groundedness: Groundedness = Field(
        ...,
        description="Evaluation of the groundedness of the generated answer, ensuring it is faithful to the retrieved context.",
    )


# Function to evaluate RAG metrics
def evaluate_rag(query: str, retrieved_context: str, generated_answer: str):
    chat_response = client.chat.completions.parse(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a judge for evaluating a Retrieval-Augmented Generation (RAG) system. "
                    "Evaluate the context relevance, answer relevance, and groundedness based on the following criteria: "
                    "Provide a reasoning and a score as a string between '0' and '3' for each criterion. "
                    "Context Relevance: How relevant is the retrieved context to the query? "
                    "Answer Relevance: How relevant is the generated answer to the query? "
                    "Groundedness: How faithful is the generated answer to the retrieved context?"
                ),
            },
            {
                "role": "user",
                "content": f"Query: {query}\nRetrieved Context: {retrieved_context}\nGenerated Answer: {generated_answer}",
            },
        ],
        response_format=RAGEvaluation,
        temperature=0,
    )
    return chat_response.choices[0].message.parsed


# Example usage
query = "What are the benefits of renewable energy?"
retrieved_context = "Renewable energy includes solar, wind, hydro, and geothermal energy, which are naturally replenished."
generated_answer = "Renewable energy sources like solar and wind are environmentally friendly and reduce carbon emissions."
evaluation = evaluate_rag(query, retrieved_context, generated_answer)
output = "🏆 RAG Evaluation:"
output += "\nCriteria: Context Relevance\n"
output += f"Reasoning: {evaluation.context_relevance.explanation}\n"
output += f"Score: {evaluation.context_relevance.score.value}/3\n"
output += "\nCriteria: Answer Relevance"
output += f"\nReasoning: {evaluation.answer_relevance.explanation}\n"
output += f"\nScore: {evaluation.context_relevance.score.value}/3\n"
output += "\nCriteria: Groundedness"
output += f"\nReasoning: {evaluation.groundedness.explanation}\n"
output += f"\nScore: {evaluation.groundedness.score.value}/3\n"
# Print the evaluation
# print("🏆 RAG Evaluation:")
# print("\nCriteria: Context Relevance")
# print(f"Reasoning: {evaluation.context_relevance.explanation}")
# print(f"Score: {evaluation.context_relevance.score.value}/3")

# print("\nCriteria: Answer Relevance")
# print(f"Reasoning: {evaluation.answer_relevance.explanation}")
# print(f"Score: {evaluation.answer_relevance.score.value}/3")

# print("\nCriteria: Groundedness")
# print(f"Reasoning: {evaluation.groundedness.explanation}")
# print(f"Score: {evaluation.groundedness.score.value}/3")

In [6]:
console.print(output)

🏆 RAG Evaluation:
Criteria: Context Relevance
Reasoning: The retrieved context provides a definition of renewable energy and lists its types, which is relevant 
to the query about the benefits of renewable energy. However, it does not directly address the benefits themselves,
which is the main focus of the query. Therefore, while it is somewhat relevant, it does not fully meet the user's 
intent.
Score: 2/3

Criteria: Answer Relevance
Reasoning: The generated answer directly addresses the query by stating that renewable energy sources are 
environmentally friendly and reduce carbon emissions, which are indeed benefits of renewable energy. This makes the
answer relevant to the user's question about the benefits.

Score: 2/3

Criteria: Groundedness
Reasoning: The generated answer is grounded in the context provided, as it mentions solar and wind energy, which 
are included in the retrieved context. The benefits mentioned in the answer (environmental friendliness and 
reduction of carbon emissions) are consistent with the characteristics of renewable energy. Thus, the answer is 
faithful to the retrieved context.

Score: 3/3

In [7]:


# Now write the file
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    f.write(output)